In [1]:
import numpy as np
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from surprise import Dataset, Reader, SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
#from surprise.model_selection import cross_validate, train_test_split
from surprise.prediction_algorithms import KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore
from surprise import accuracy
from surprise import dump
from sklearn.preprocessing import normalize

In [3]:
movies_df = pd.read_csv("D:\\Year 3 Semester 2\\computerScienceProject\\tmdb_5000_movies.csv")
credits_df = pd.read_csv("D:\\Year 3 Semester 2\\computerScienceProject\\tmdb_5000_credits.csv")
ratings_df = pd.read_excel("D:\\Year 3 Semester 2\\computerScienceProject\\movie_ratings.xlsx")

In [4]:
# book_ratings_df = pd.read_excel("D:\Year 3 Semester 2\computerScienceProject\BX-Book-Ratings.xlsx")
# books_df = pd.read_csv("D:\Year 3 Semester 2\computerScienceProject\BX-Books.csv", sep=";", error_bad_lines=False, encoding="latin-1")
# books_df.columns = ["ISBN","Title","Author","Year_Of_Publication","Publisher","Image-URL-S","Image-URL-M","Image-URL-L"]

In [5]:
#merge the two datasets
movies_df = movies_df.merge(credits_df, on='title')

In [6]:
movies_df = movies_df.merge(ratings_df, on='movie_id')

In [7]:
#selecting relevant columns
movies_final = movies_df[[ 'id', 'title', 'genres', 'runtime', 'overview', 'keywords', 'cast', 'crew', 'User_ID', 'RATINGS']]

In [8]:
#check for null values
print(movies_final.isna().sum())

id          0
title       0
genres      0
runtime     2
overview    3
keywords    0
cast        0
crew        0
User_ID     0
RATINGS     1
dtype: int64


In [9]:
#drop null values
movies_final.dropna(inplace=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\326592079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final.dropna(inplace=True)


In [10]:
# books_df = books_df.merge(book_ratings_df, on='ISBN')

In [11]:
# books_final = books_df[['ISBN', 'Title', 'Author', 'Year_Of_Publication', 'Publisher', 'User_ID', 'Rating']]

In [12]:
# print(books_final.isna().sum())

In [13]:
#function to extract genre name
def convert(obj):
    l = []

    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

movies_final['genres'] = movies_final['genres'].apply(convert)


C:\Users\HP\AppData\Local\Temp\ipykernel_14844\2571114438.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['genres'] = movies_final['genres'].apply(convert)


In [14]:
movies_final['keywords'] = movies_final['keywords'].apply(convert)

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\190040370.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['keywords'] = movies_final['keywords'].apply(convert)


In [15]:
#function to extract first three cast names
def convert3(obj):
    l = []
    counter = 0

    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter += 1
        else:
            break
    return l

movies_final['cast'] = movies_final['cast'].apply(convert3)


C:\Users\HP\AppData\Local\Temp\ipykernel_14844\65014636.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['cast'] = movies_final['cast'].apply(convert3)


In [16]:
# to extract director's name
def extract_director(obj):
    l = []

    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

movies_final['crew'] = movies_final['crew'].apply(extract_director)

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\550475311.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['crew'] = movies_final['crew'].apply(extract_director)


In [17]:
movies_final['overview'] = movies_final['overview'].apply(lambda x: x.split()) 

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\2814892025.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['overview'] = movies_final['overview'].apply(lambda x: x.split())


In [18]:
# removing spaces between words
movies_final['genres'] = movies_final['genres'].apply(lambda x : [i.replace(" ", "") for i in x])
movies_final['keywords'] = movies_final['keywords'].apply(lambda x : [i.replace(" ", "") for i in x])
movies_final['cast'] = movies_final['cast'].apply(lambda x : [i.replace(" ", "") for i in x])
movies_final['crew'] = movies_final['crew'].apply(lambda x : [i.replace(" ", "") for i in x])

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\361890825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['genres'] = movies_final['genres'].apply(lambda x : [i.replace(" ", "") for i in x])
C:\Users\HP\AppData\Local\Temp\ipykernel_14844\361890825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['keywords'] = movies_final['keywords'].apply(lambda x : [i.replace(" ", "") for i in x])
C:\Users\HP\AppData\Local\Temp\ipykernel_14844\361890825.py:4: SettingWithCopyWarning: 
A valu

In [19]:
movies_final['tags'] = movies_final['genres'] + movies_final['keywords'] + movies_final['cast'] + movies_final['crew'] + movies_final['overview']

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\451815744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_final['tags'] = movies_final['genres'] + movies_final['keywords'] + movies_final['cast'] + movies_final['crew'] + movies_final['overview']


In [20]:
# books_final['tags'] = books_final['Author'] + books_final['Publisher']

In [21]:
movies = movies_final[['id', 'title', 'tags', 'User_ID', 'RATINGS']]

In [22]:
# books = books_final[['ISBN', 'Title', 'tags', 'User_ID', 'Rating']]

In [23]:
movies['tags'] = movies['tags'].apply(lambda x : " ".join(x))

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\3007261697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x : " ".join(x))


In [24]:
movies['tags'] = movies['tags'].apply(lambda x : x.lower())

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\2861373371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['tags'] = movies['tags'].apply(lambda x : x.lower())


In [25]:
# # Assuming 'books' is your DataFrame containing ratings data
# # Step 1: Check unique values in the 'Rating' column
# print(books['Rating'].unique())

# # Step 2: Handle non-numeric values
# # For example, if '"' is a non-numeric value, you can replace it with NaN
# books['Rating'] = pd.to_numeric(books['Rating'], errors='coerce')

# # Step 3: Convert the 'Rating' column to float
# books['Rating'] = books['Rating'].astype(float)

# # Now you can load the data into Surprise's Dataset
# reader = Reader(rating_scale=(1, 10))
# data = Dataset.load_from_df(books[['ISBN', 'User_ID', 'Rating']], reader)

In [25]:
movies['RATINGS'] = movies['RATINGS'].replace(0, 1)

C:\Users\HP\AppData\Local\Temp\ipykernel_14844\1226471429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies['RATINGS'] = movies['RATINGS'].replace(0, 1)


In [26]:
movies

,id,title,tags,User_ID,RATINGS
0,19995,Avatar,action adventure fantasy sciencefiction cultur...,227945,10.0
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drugabuse exoti...,250634,10.0
2,206647,Spectre,action adventure crime spy basedonnovel secret...,250634,10.0
3,49026,The Dark Knight Rises,action crime drama thriller dccomics crimefigh...,250634,10.0
4,49529,John Carter,action adventure sciencefiction basedonnovel m...,250634,10.0
...,...,...,...,...,...
4803,67238,Cavite,"foreign thriller neilldelallana adam, a securi...",244349,1.0
4804,9367,El Mariachi,action crime thriller unitedstates–mexicobarri...,244349,1.0
4805,72766,Newlyweds,comedy romance edwardburns kerrybishé marshadi...,244349,1.0
4806,231617,"Signed, Sealed, Delivered",comedy drama romance tvmovie date loveatfirsts...,244341,1.0


In [36]:
reader = Reader(rating_scale = (1, 10))
data = Dataset.load_from_df(movies[['id', 'User_ID', 'RATINGS']], reader)
traindf = data.build_full_trainset()
testdf = traindf.build_anti_testset()
# # testdf = filter(lambda x : x[0] == User_ID, testdf)

In [49]:
# basic collaborative filtering algorithms
sim_options = {'name' : 'cosine',
               'user_based' : False  #defining similarity measures
              }
               
knnbaseline_model = KNNBaseline(sim_options=sim_options)

knnbaseline_model.fit(traindf)
knnbaseline_predictions = knnbaseline_model.test(testdf)
accuracy.rmse(knnbaseline_predictions)
accuracy.mae(knnbaseline_predictions)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.


MemoryError: 

In [30]:
# svd_model = SVD()

# svd_model.fit(traindf)
# svd_model_predictions = svd_model.test(testdf)

# accuracy.rmse(svd_model_predictions)
# accuracy.mae(svd_model_predictions)

In [37]:
svdpp_model = SVDpp()

svdpp_model.fit(traindf)
svdpp_model_predictions = svdpp_model.test(testdf)

accuracy.rmse(svdpp_model_predictions)
accuracy.mae(svdpp_model_predictions)

RMSE: 0.5509
MAE:  0.4447


0.4447363814175227

In [38]:
#  content-based filtering
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
content_matrix = tfidf_vectorizer.fit_transform(movies['tags'])
content_similarity = linear_kernel(content_matrix, content_matrix)

In [ ]:
# def get_content_based_recs(id, top_n):
#     index = movies[movies['id'] == id].index[0]
#     similarity_scores = content_similarity[index]
#     similar_indices = similarity_scores.argsort()[::-1][1:top_n + 1]
#     recommendations = movies.loc[similar_indices, 'id']
#     return recommendations

In [ ]:
# #second, collaborative filtering
# collab_model = SVD()
# trainset = data.build_full_trainset()
# collab_model.fit(trainset)

In [ ]:
# def get_collab_recs(User_ID, top_n):
#     testset = trainset.build_anti_testset()
#     testset = filter(lambda x : x[0] == User_ID, testset)
#     predictions = collab_model.test(testset)
#     predictions.sort(key=lambda x : x.est, reverse=True)
#     recommendations = [predictions.iid for prediction in predictions[:top_n]]
#     return recommendations

In [ ]:
# #now, hybrid filtering
# def get_hybrid_recs(id, User_ID, top_n):
#     content_based_recs = get_content_based_recs(id, top_n)
#     collaborative_recs = get_collab_recs(User_ID, top_n)
#     hybrid_recs = list(set(content_based_recs + collaborative_recs))
#     return hybrid_recs[:top_n]

In [ ]:
# #tryout
# User_ID = 2408
# id = 285
# top_n = 10
# recommendations = get_hybrid_recs(id, User_ID, top_n)

# print(f"Hybrid Recommendations for User {User_ID} based on Product {id}:")
# for i, recommendation in enumerate(recommendations):
#     print(f"{i + 1}. Product ID: {recommendation}")

In [39]:
mapping = pd.Series(movies.index, index=movies['title'])

In [40]:
#recommender function based on metadata
def metadata_recs(movie_input):
    index = mapping[movie_input]
    
    #get similarity with other movies
    similarity_score = list(enumerate(content_similarity[index]))
    similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
    
    # Get the scores of the 15 most similar movies. Ignore the first movie.
    similarity_score = similarity_score[1:15]
    indices = [i[0] for i in similarity_score]
    return (movies['title'].iloc[indices])

In [41]:
content_recs = metadata_recs('Tangled')
content_recs

2315                           Out of Inferno
39                               TRON: Legacy
3136                        Of Horses and Men
1701                                  Aladdin
330     The Lord of the Rings: The Two Towers
4174                              Man on Wire
3998                       The Black Stallion
269                 The Princess and the Frog
3889                                    Feast
950                            The Negotiator
1990                The Thief and the Cobbler
4665                               Horse Camp
403                          Last Action Hero
1476                                   Stolen
Name: title, dtype: object

In [ ]:

# def hybrid(User_ID):
#     user_movies = test_data[test_data['User_ID'] == User_ID]
#     user_movies['est'] = user_movies['id'].apply(lambda x: 0.6*svdpp_model.predict(User_ID, x).est + 0.4*metadata_recs.predict(User_ID, x).est)    
#     user_movies = user_movies.sort_values(by ='est', ascending=False).head(4)
#     user_movies['Model'] = 'svdpp + cb'
# #     user_movies = user_movies['movieId'].values.tolist()
# #     print("User liked movies list: ", user_movies)
    
#     recommend_list = user_movies[['id', 'est', 'Model']]
#     print(recommend_list.head())
#     return recommend_list

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.neighbors import NearestNeighbors

# class HybridRecommendationSystem:
#     def __init__(self, user_item_matrix, item_features):
#         self.user_item_matrix = user_item_matrix
#         self.item_features = item_features

#     def collaborative_filtering(self, user_id, top_n):
#         # Compute similarity between users using cosine similarity
#         user_similarity = cosine_similarity(self.user_item_matrix)
        
#         # Get top N similar users
#         similar_users = user_similarity[user_id].argsort()[::-1][:top_n]

#         # Aggregate recommendations from similar users
#         recommendations = []
#         for user in similar_users:
#             user_ratings = self.user_item_matrix[user]
#             for item, rating in enumerate(user_ratings):
#                 if rating > 0 and self.user_item_matrix[user_id][item] == 0:
#                     recommendations.append(item)
#                     if len(recommendations) >= top_n:
#                         return recommendations
        
#         return recommendations

#     def content_based_filtering(self, user_id, top_n):
#         # Convert item features into TF-IDF vectors
#         tfidf_vectorizer = TfidfVectorizer()
#         tfidf_matrix = tfidf_vectorizer.fit_transform(self.item_features)

#         # Compute similarity between items based on their features
#         item_similarity = cosine_similarity(tfidf_matrix)

#         # Get top N similar items
#         similar_items = item_similarity.argsort()[::-1][user_id][:top_n]

#         return similar_items

#     def hybrid_recommendation(self, user_id, top_n):
#         collaborative_rec = self.collaborative_filtering(user_id, top_n)
#         content_based_rec = self.content_based_filtering(user_id, top_n)

#         # Combine recommendations, for example, by merging the lists and removing duplicates
#         hybrid_rec = collaborative_rec + content_based_rec
#         hybrid_rec = list(set(hybrid_rec))  # Remove duplicates
        
#         return hybrid_rec[:top_n]


In [46]:
# def combine_recommendations(svdpp_model_predictions, content_recs, weight_collab=0.5):
#     # Here you can implement a strategy to combine the recommendations
#     # For example, weighted average:
#     hybrid_scores = weight_collab * svdpp_model_predictions + (1 - weight_collab) * content_recs
def combine_recommendations(svdpp_model_predictions, content_recs, weight_collab=0.5):
    # Convert svdpp_model_predictions to a list of recommended items
    svdpp_items = [prediction.iid for prediction in svdpp_model_predictions]
    
    # Combine recommendations using the specified weight
    hybrid_scores = []
    for prediction in svdpp_model_predictions:
        for content_score in content_recs:
           hybrid_scores.append((weight_collab * prediction.est) + ((1 - weight_collab) * content_score))
    
    # Combine the recommended items from collaborative and content-based filtering
    hybrid_items = svdpp_items * len(content_recs)
    
    # Combine the scores and items into tuples
    hybrid = list(zip(hybrid_items, hybrid_scores))
    
    # Sort the hybrid recommendations by score in descending order
    hybrid.sort(key=lambda x: x[1], reverse=True)
    
    # Extract only the recommended items from the sorted hybrid list
    hybrid = [item for item, _ in hybrid]
    
    return hybrid

In [47]:
def hybrid_recs(movie_input):
#     collab = svdpp_model
#     content = metadata_recs(movie_input)
    
#     hybrid = collab + content
#     hybrid = list(set(hybrid))
      hybrid = combine_recommendations(svdpp_model_predictions, content_recs)
    
      return hybrid[:top_n]

In [45]:
combine_recommendations(svdpp_model_predictions, content_recs)

TypeError: can't multiply sequence by non-int of type 'float'

In [48]:
hybrid_recs('Avatar')

TypeError: can't multiply sequence by non-int of type 'float'